In [1]:
import tensorflow as tf
import numpy as np

In [44]:
class AutoEncoder(object):
    
    def __init__(self, input_dim, hidden_dim, lr, epochs, bs):
        
        self.lr = lr
        self.epochs = epochs
        self.bs = bs
        
        self.x = tf.placeholder(dtype=tf.float32, shape=[None, input_dim], name='x')
        
        with tf.name_scope('encoder'):
            weights = tf.Variable(tf.random_normal([input_dim, hidden_dim]), name='weights')
            biases = tf.Variable(tf.zeros([hidden_dim]), name='biases')
            encode = tf.nn.tanh(tf.matmul(self.x, weights) + biases)
        
        with tf.name_scope('decoder'):
            weights = tf.Variable(tf.random_normal([hidden_dim, input_dim]), name='weights')
            biases = tf.Variable(tf.zeros([input_dim]), name='biases')
            decode = tf.matmul(encode, weights) + biases
            
        self.encode = encode
        self.decode = decode
        
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decode))))
        self.train_op = tf.train.RMSPropOptimizer(self.lr).minimize(self.loss)
        self.saver = tf.train.Saver()
        
    def train(self, data):
        num_samples = len(data)
        num_batches = int(num_samples / self.bs)
        
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epochs):
                for j in range(num_batches):
                    batch = data[np.random.choice(num_samples, self.bs, replace= False)]
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x: batch})
                if i%10 == 0:
                    print('epoch: {0}, loss={1}'.format(i, l))
                
            self.saver.save(sess, 'autoencoder_model.ckpt')
            
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess, './autoencoder_model.ckpt')
            hidden, reconstructed = sess.run([self.encode, self.decode], feed_dict={self.x: data})
        
        print( 'input ', data)
        print( 'compressed' , hidden)
        print( 'reconstructed' , reconstructed)
        return reconstructed

In [48]:
from sklearn import datasets

hidden_dim=2
data = datasets.load_iris().data
input_dim = len(data[0])
ae = AutoEncoder(input_dim, hidden_dim, 0.01, 300, 10)
ae.train(data)

epoch: 0, loss=4.402854919433594
epoch: 10, loss=1.5729666948318481
epoch: 20, loss=0.6488809585571289
epoch: 30, loss=0.4366246461868286
epoch: 40, loss=0.3858313262462616
epoch: 50, loss=0.41962137818336487
epoch: 60, loss=0.3147802948951721
epoch: 70, loss=0.40170150995254517
epoch: 80, loss=0.2726631462574005
epoch: 90, loss=0.26577094197273254
epoch: 100, loss=0.3993598222732544
epoch: 110, loss=0.3006899654865265
epoch: 120, loss=0.2344166785478592
epoch: 130, loss=0.3055359423160553
epoch: 140, loss=0.32929331064224243
epoch: 150, loss=0.30390119552612305
epoch: 160, loss=0.21616226434707642
epoch: 170, loss=0.2644341289997101
epoch: 180, loss=0.1692265421152115
epoch: 190, loss=0.23742546141147614
epoch: 200, loss=0.23149912059307098
epoch: 210, loss=0.2001902163028717
epoch: 220, loss=0.2280314564704895
epoch: 230, loss=0.2998230755329132
epoch: 240, loss=0.26532673835754395
epoch: 250, loss=0.19614939391613007
epoch: 260, loss=0.28185975551605225
epoch: 270, loss=0.1764996796

In [49]:
ae.test([[8, 4, 6, 2]])

INFO:tensorflow:Restoring parameters from ./autoencoder_model.ckpt
input  [[8, 4, 6, 2]]
compressed [[-0.96621865 -0.75487101]]
reconstructed [[ 7.13904095  3.483845    5.89659595  2.07307124]]


array([[ 7.13904095,  3.483845  ,  5.89659595,  2.07307124]], dtype=float32)